In [2]:
!pip install langdetect
!pip install gensim
import numpy as np
import pandas as pd
import csv
import langdetect
import gensim.models
from gensim.models import Word2Vec
from keras.models import model_from_json
import pickle
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
np.random.seed(0)
#from keras.models import Model
#from keras.layers import Dense, Input, Dropout, LSTM, Activation
#from keras.layers.embeddings import Embedding
#from keras.preprocessing import sequence
#from keras.initializers import glorot_uniform
np.random.seed(1)

%pylab inline
#read tweets
folder = "/Users/hovensa/OneDrive - delaware/Data science/WK/"
#df1 = pd.read_csv(folder + "rodeduivels_2018-06-03.csv", delimiter = ';')
#df2 = pd.read_csv(folder + "#PorBel_2018-06-06.csv", delimiter = ';')
#df3 = pd.read_csv(folder + "#BelPor_2018-06-06.csv", delimiter = ';')
#df4 = pd.read_csv(folder + "DiablesRouges_2018-06-03.csv", delimiter = ';')
#df5 = pd.read_csv(folder + "RedDevils_2018-06-03.csv", delimiter = ';')
#df6 = pd.read_csv(folder + "DS_Twitter_KAAGent.csv", delimiter = ';')
#df7 = pd.read_csv(folder + "DS_Twitter_RSCA.csv", delimiter = ';')

df1 = pd.read_csv(folder + "#BELBRA_2018-07-10.csv", delimiter = ';')
df2 = pd.read_csv(folder + "#DiablesRouges_2018-07-10.csv", delimiter = ';')
df3 = pd.read_csv(folder + "#RodeDuivels_2018-07-10.csv", delimiter = ';')
df4 = pd.read_csv(folder + "#REDTOGETHER_2018-07-10.csv", delimiter = ';')
#df5 = pd.read_csv(folder + "#BRABEL_2018-07-10.csv", delimiter = ';')





dfTemp = [df1,df2,df3,df4]
df = pd.concat(dfTemp, ignore_index="True")  
df_url = df.text[:].str.replace('http\S+|www.\S+', '', case=False)
df_RT = df_url.str.replace('RT ','',case=True)
df_tags = df_RT.str.replace('(@\w{0,})', '',case=False)
df_hash = df_tags.str.replace('(#\w{0,})', '',case=False)
df_col2 = df_hash.str.replace('FACEBOOK LIVE Q&amp;A: ','',case=False)
df_trim = df_col2.str.lstrip()

df_final = pd.DataFrame({"created_at":df.created_at, "text": df_trim})


Populating the interactive namespace from numpy and matplotlib


In [3]:
df_text = pd.DataFrame(df_final)
df_text['Language'] = ''

for index,i in df_text.iterrows():
    try: 
        if langdetect.detect(i.text) == 'af':
            i.Language = 'nl'
        if langdetect.detect(i.text) != 'nl' and langdetect.detect(i.text) != 'fr':
            i.Language = 'NA'
        else: 
            i.Language = langdetect.detect(i.text)
            
    except:
        i.Language = 'NA'

df_text
df_filtered = df_text[df_text['Language'] != 'NA'] 

In [4]:
def DefineEmoticons(language,text):
    
    emoticondf = pd.read_csv(open(folder + "emoticons_"+language+".txt",'rt'), delimiter = "\t")
    text["emotion_"+language] = ""
    for index,line in text.iterrows():
        for index,row in emoticondf.iterrows():
            if(row.Emoticon in line.text): 
                line["emotion_"+language] = row.Description
           
    return text

text_ad = DefineEmoticons("nl",df_filtered)
text_ad = DefineEmoticons("en",text_ad)       
text_ad = DefineEmoticons("fr",text_ad)  

C:\Users\hovensa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
#the addition of an emotion word to the tweet text
text_ad['emotion']= ''
for index,i in text_ad.iterrows():

        if i.Language == 'nl':
            i.emotion = i.emotion_nl
        if i.Language == 'fr':
            i.emotion = i.emotion_fr
        if i.Language == 'en':
            i.emotion = i.emotion_en
        else:
            '' 
            
text_ad['text'] = text_ad['text'].map(str) + ' '+ text_ad['emotion'] 

C:\Users\hovensa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\hovensa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
for index,tweet in text_ad.iterrows():
    if tweet.Language == 'nl':
        tweet.text = tweet.text.replace('!',' uitroepteken')
        tweet.text = tweet.text.replace('?',' vraagteken')
        text_ad.loc[index,:].text = tweet.text
    else: 
        if tweet.Language == 'fr':
            tweet.text = tweet.text.replace('!', ' point d\'exclamation')
            tweet.text = tweet.text.replace('?',' point d\'interrogation')
            text_ad.loc[index,:].text = tweet.text

        else: 
            tweet.text = tweet.text.replace('!', ' exclamation mark')
            tweet.text = tweet.text.replace('?',' question mark')
            text_ad.loc[index,:].text = tweet.text


In [7]:
def word2Vec(language,df):
    dataSet = df[df.Language == language]
    model = gensim.models.Word2Vec.load(folder + language + '.bin')  # you can continue training with the loaded model!    
    #create the appropriate format to plot the tweets in a vectorspace
    #punctuation can be deleted now since the emotions are already converted
    sentences = []
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for word in dataSet.text: 
        no_punct = ""
        for char in word:
            if char not in punctuations:
                no_punct = no_punct + char
        sentence = no_punct.lower().split()
        sentences.append(sentence)

    #store each vector representation of a word of a tweet in a list 'vectors'
    vectors = []
    vocab = model.wv.vocab
    for w in sentences:
        vector = []
        for i in w:
            if i in vocab:
                vector.append(model[i])

        vectors.append(vector)
    return vectors


In [8]:
#create a function to sum lists
def sum_list(l):
    sum = 0
    for x in l:
        sum += x
    return sum    
def TakeVectorAverage(VectorList):
    #to take the average, we need to know how many words each tweet has
    count = []
    for tweet in VectorList:
        count.append(len(tweet))
    #sum up each list tp a total that can be divided by the count of the previous step
    vectorsAvg = []
    for i in range(len(count)):
        vectorSum = []
        vector = VectorList[i]
        if len(vector)>1:        
            for j in range(len(vector)):
                vectorSum.append(vector[j])
            total = sum_list(vectorSum)
            average = total/count[i]
            vectorsAvg.append(average)
        else:
            vectorsAvg.append(0)
    return vectorsAvg

In [9]:
#get the indices and create padded lists for each tweet
def tweet_embedding_and_indexing(df,word_vec,MaxLenTweet):
    
    #flatten the word_vector list in order to be usable in the following steps
    flat_list = [item for sublist in word_vec for item in sublist]
    
    #initiate the tokenizer --> is used to assign indeces to the words in a tweet
    t = Tokenizer()
    t.fit_on_texts(df.text)
    
    #how many words can be found in the dataset --> not taking into account those that are not found in the word2vec mechanism 
    vocab_size = len(flat_list)
    #split the tweets into words
    encoded_docs = t.texts_to_sequences(df.text)

    #fill the tweet lists until max_length is reahced with 0 --> necessary for the creation of the embedding layer
    padded_docs = pad_sequences(encoded_docs, maxlen= MaxLenTweet, padding='post')
    
    #create the embedding layer with the weights based on the vector space
    e = Embedding((vocab_size), 300, weights= [np.array(flat_list)], input_length= MaxLenTweet, trainable=False)
    
    return padded_docs,e

In [10]:
def Dataset(language,df):
    tweetVectors = []
    yVar = []
    MaxLenTweet = 40#max(df.text.apply(lambda x : len(x.split())))
    #filter each language subgroup
    vectors = word2Vec(language,df)
    #Emotion = text_ad[text_ad.Language == language].Emotion
    TestIndices,_  = tweet_embedding_and_indexing(df,vectors,MaxLenTweet)
    ###vectors = word2Vec('en',text_ad)
    ##tweetVectors.append(vectors)
    TweetVectorAvg = TakeVectorAverage(vectors)
    #final = pd.DataFrame({"WordVector": TweetVectorAvg,"Emotion": Emotion.tolist()})
    final = pd.DataFrame({"WordVector": TweetVectorAvg})
    
    #we want to have all vectors in separate columns
    dataf = pd.DataFrame()
    dataf = pd.DataFrame(final.WordVector[0]).T
    for i in range(1,len(final.WordVector)):
        dataf.loc[len(dataf)] = final.WordVector.loc[i]
    #dataf['Emotion'] = final.Emotion
    #dataf .to_csv('WKDataset ' + language + '.csv', sep=';', encoding='utf-8')
    
    return dataf,TestIndices


In [11]:
def ObtainPred(model,DataSet, sentimentClasses):
    prediction = []
    if(len(DataSet)>0):
        pred = model.predict(DataSet)
        prediction = []
        if (pred[0][1] <1): 
            classesT = pred
            for j in range(len(classesT)):
                    classesT[j][classesT[j] == classesT[j].max()] = 1
                    classesT[j][classesT[j] != classesT[j].max()] = 0 

            yDF = pd.DataFrame(classesT)
            y = yDF.apply(lambda x: int(np.argmax(x)), axis=1)
            NrClasses = len(sentimentClasses)
            ConvertClass = dict()
            for i in range(NrClasses):
                ConvertClass[sentimentClasses[i]] = i

            bDict = {val:key for (key, val) in ConvertClass.items()}
            yDF = pd.DataFrame(classesT)
            y = yDF.apply(lambda x: int(np.argmax(x)), axis=1)
           
            y = y.replace(bDict,regex = True)
            y = y.replace(bDict,regex = True)
            prediction = y
            print(prediction)
        else: 
            prediction = pred
    else:
        prediction = []
    return prediction

In [12]:
DatasetNL,IndicesNl = Dataset('nl',text_ad)
DatasetFR, IndicesFr = Dataset('fr',text_ad)

C:\Users\hovensa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [13]:
filenameNL = folder + "modelnl.json"
filenameFR = folder + "modelfr.json"
#modelNL = pickle.load(open(filenameNL, 'rb'))
#modelFR = pickle.load(open(filenameFR, 'rb'))

In [14]:
json_file = open(filenameNL, 'r')
loaded_modelLSTMNl_json = json_file.read()
json_file.close()
loaded_modelNl = model_from_json(loaded_modelLSTMNl_json)

In [15]:
json_file = open(filenameFR, 'r')
loaded_modelFr_json = json_file.read()
json_file.close()
loaded_modelFr = model_from_json(loaded_modelFr_json)

In [16]:
PredictionNL = ObtainPred(loaded_modelNl, DatasetNL,["anger","cynicism","dissapointment","excitement","hope","neutral"])
PredictionFR = ObtainPred(loaded_modelFr, DatasetFR, ["anger","cynicism","dissapointment","excitement","hope","neutral"])

0              hope
1              hope
2        excitement
3          cynicism
4        excitement
5        excitement
6              hope
7          cynicism
8          cynicism
9        excitement
10         cynicism
11         cynicism
12             hope
13       excitement
14         cynicism
15             hope
16            anger
17            anger
18            anger
19         cynicism
20       excitement
21         cynicism
22       excitement
23       excitement
24             hope
25         cynicism
26             hope
27       excitement
28             hope
29       excitement
            ...    
11217         anger
11218    excitement
11219    excitement
11220      cynicism
11221      cynicism
11222      cynicism
11223      cynicism
11224      cynicism
11225          hope
11226    excitement
11227      cynicism
11228         anger
11229      cynicism
11230      cynicism
11231      cynicism
11232      cynicism
11233      cynicism
11234    excitement
11235    excitement


In [17]:
DfFr = text_ad[text_ad.Language=='fr'].reset_index(drop=True)

In [18]:
DfNl = text_ad[text_ad.Language == 'nl'].reset_index(drop=True)

In [19]:
FinalDataNL = pd.DataFrame({"created_at":DfNl.created_at, "text": DfNl.text, "prediction":PredictionNL,"language":DfNl.Language})
FinalDataFR = pd.DataFrame({"created_at":DfFr.created_at, "text": DfFr.text, "prediction":PredictionFR,"language":DfFr.Language})
#FinalDataNL.to_csv('FinalDataSet ' + 'nl' + '.csv', sep=';', encoding='utf-8')
#FinalDataFR.to_csv('FinalDataSet ' + 'fr' + '.csv', sep=';', encoding='utf-8')

In [20]:
!pip install pypyodbc

In [21]:
!pip install sqlalchemy

In [22]:
from sqlalchemy import create_engine

In [23]:
import pypyodbc
from sqlalchemy import create_engine
import urllib 

In [24]:
server = 'ikulexdfcr.database.windows.net'
database = 'DEMO_WK2018'
username = 'demo'
password = '{Delaware123;}'

In [25]:

params = urllib.parse.quote('Driver={ODBC Driver 13 for SQL Server};SERVER='+server+';PORT=1443;Trusted_Connection=no;DATABASE='+database+';UID='+username+';PWD='+password+';Encrypt=yes;TrustServerCertificate=no;Persist Security Info = True') 
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params) 
conn = engine.connect() 
tablename = 'TweetsBELJAP'

# Push the data
FinalDataNL.to_sql(tablename, con=engine, if_exists='replace') 
FinalDataFR.to_sql(tablename, con=engine, if_exists='append') 

Probeer tot hier eens

In [ ]:
PredictionFR = loaded_modelFr.predict(DatasetFR)

In [ ]:
PredictionFR[2:]

In [ ]:
classesT = PredictionNL
for j in range(len(classesT)):
        classesT[j][classesT[j] == classesT[j].max()] = 1
        classesT[j][classesT[j] != classesT[j].max()] = 0 

In [ ]:
classesT[1:30]

In [ ]:
NrClasses = len()
ConvertClass = dict()
for i in range(NrClasses):
    ConvertClass[classes[i]] = i

In [ ]:
#switch key and value, in order to map words wuth columnindex                
bDict = {val:key for (key, val) in ConvertClass.items()}
yDF = pd.DataFrame(classesT)
y = yDF.apply(lambda x: int(np.argmax(x)), axis=1)
print(y)
y = y.replace(bDict,regex = True)

In [43]:
FinalDataNL["created_at"] =text_ad[text_ad.Language=='nl'].created_at

In [45]:
FinalDataNL["text"] = text_ad[text_ad.Language=='fr'].text

In [41]:
FinalDataNL = pd.DataFrame()

In [34]:
Test

[0         cynicism
 1         cynicism
 2         cynicism
 3            anger
 4         cynicism
 5       excitement
 6            anger
 7             hope
 8            anger
 9         cynicism
 10        cynicism
 11         neutral
 12        cynicism
 13        cynicism
 14        cynicism
 15        cynicism
 16        cynicism
 17        cynicism
 18      excitement
 19      excitement
 20         neutral
 21      excitement
 22         neutral
 23        cynicism
 24        cynicism
 25      excitement
 26        cynicism
 27           anger
 28        cynicism
 29        cynicism
            ...    
 1198          hope
 1199    excitement
 1200      cynicism
 1201      cynicism
 1202      cynicism
 1203      cynicism
 1204          hope
 1205      cynicism
 1206          hope
 1207         anger
 1208      cynicism
 1209      cynicism
 1210      cynicism
 1211      cynicism
 1212      cynicism
 1213      cynicism
 1214    excitement
 1215    excitement
 1216      cynicism


In [22]:
FinalDataNL

,created_at,language,prediction,text
0,NaN,NaN,cynicism,NaN
1,NaN,NaN,cynicism,NaN
2,NaN,NaN,cynicism,NaN
3,NaN,NaN,anger,NaN
4,NaN,NaN,cynicism,NaN
5,NaN,NaN,excitement,NaN
6,NaN,NaN,anger,NaN
7,NaN,NaN,hope,NaN
8,NaN,NaN,anger,NaN
9,NaN,NaN,cynicism,NaN
